# Import packages

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tensorflow.keras import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Flatten,Dense,MaxPooling2D,Conv2D,Dropout
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam, SGD
import os
import random
import time
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


# Import train, validation, and test datasets

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_normalize = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_normalize = ImageDataGenerator(rescale=1./255)
val_normalize = ImageDataGenerator(rescale=1./255)

main_dir = 'splits/split2_new/'
train_set = train_normalize.flow_from_directory(directory=main_dir+"train",target_size=(225,225),class_mode='categorical')
val_set = val_normalize.flow_from_directory(directory=main_dir+"val",target_size=(225,225),class_mode="categorical")
test_set = test_normalize.flow_from_directory(directory=main_dir+"test",target_size=(225,225),class_mode="categorical")

fig_size = 225
checkpoint_dir = 'checkpoints/Optimization'

# Compile Model

In [ ]:
def create_model(optimizer = 'adam',learning_rate=0.001):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), input_shape=(fig_size, fig_size, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation='softmax'))
    # if optimizer == 'adam':

    learning_rate = 0.001
    opt = Adam(learning_rate=learning_rate)
    # else :
    # opt = SGD(learning_rate=learning_rate, momentum=0.9) #lol
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model



# Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier


batch_sizes = reversed([8 , 32])
epochs = 25
#learning_rates = [0.001,0.01, 0.1]
# learning_rate = 0.001
#optimizers = ['adam', 'sgd']

# (X_train,Y_train) = train_set
# model = KerasClassifier(build_fn=create_model, verbose=0)
# print(model.get_params().keys())
#,learning_rate=learning_rates,
#param_grid = dict( optimizer = optimizers,batch_size=batch_sizes)
# param_grid = dict( batch_size=batch_sizes)

# grid = GridSearchCV(estimator=model,param_grid=param_grid, n_jobs=3,cv=3)
# grid_result = grid.fit(X_train,Y_train)
# model = create_model()
histories = []
for batch_size in batch_sizes:
    model = create_model()
    checkpoint_path = os.path.join(checkpoint_dir,"MODEL4-BatchSize"+str(batch_size)+"-{epoch:1d}.weights.h5")
    os.makedirs(checkpoint_dir,exist_ok=True)
    checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,  # Save only the weights
        save_freq='epoch', # Save for every epoch,
        save_best_only = True
    )
    histories=(model.fit(train_set, epochs=epochs, batch_size=batch_size, validation_data=val_set, callbacks=[checkpoint_callback]))
    


# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']


# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Custom tick formatter function
def format_tick(x, pos):
    return int(x + 1)
# Plot training accuracy and loss

for history in histories:
    plt.figure(figsize=(12, 6), constrained_layout=True)
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy', color='red')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.title('Training and Validation Accuracy')
    plt.grid(True)  # Add grid
    plt.xticks(range(0, epochs+1))  # Set x-axis ticks to every integer value
    plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(format_tick))  # Apply custom tick formatter
    plt.tight_layout()  # Adjust layout
    plt.legend()

    # Plot validation accuracy and loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss', color='red')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)  # Add grid
    plt.xticks(range(0, epochs+1))  # Set x-axis ticks to every integer value
    plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(format_tick))  # Apply custom tick formatterplt.tight_layout()  # Adjust layout
    plt.show()
    plt.tight_layout(pad=5.0)
    plt.subplots_adjust(left=0.1, bottom=0.1,                    top=0.9, wspace=0.4,hspace=0.4)
    pass

# Metrics:

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_normalize = ImageDataGenerator(rescale=1./255)
test_normalize = ImageDataGenerator(rescale=1./255)
val_normalize = ImageDataGenerator(rescale=1./255)

main_dir = 'splits/split2_new/'
train_set = train_normalize.flow_from_directory(directory=main_dir+"train",target_size=(225,225),class_mode='categorical', shuffle=False)
val_set = val_normalize.flow_from_directory(directory=main_dir+"val",target_size=(225,225),class_mode="categorical", shuffle=False)
test_set = test_normalize.flow_from_directory(directory=main_dir+"test",target_size=(225,225),class_mode="categorical", shuffle=False)

fig_size = 225
checkpoint_dir = 'checkpoints/Optimization'

filepath = os.path.join(checkpoint_dir,f"MODEL4-BatchSize{32}-{23}.weights.h5")
model = create_model()
model.load_weights(filepath)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


# Calculate metrics based on confusion matrix

data_sets = [train_set,val_set,test_set] 
data_sets_names = ["Train","Validation","Test"]
for dataset in data_sets:

    predicted_labels = np.argmax(model.predict(dataset), axis=1)
    accuracy = (accuracy_score(dataset.classes, predicted_labels))
    recall = (recall_score(dataset.classes, predicted_labels, average='macro'))
    precision =(precision_score(dataset.classes, predicted_labels, average='macro'))
    f1 = (f1_score(dataset.classes, predicted_labels, average='macro'))

    print(data_sets_names[data_sets.index(dataset)])
    print(f"Accuracy :{accuracy}")
    print(f"Recall :{recall}")
    print(f"Precision :{precision}")
    print(f"F1 :{f1}")
